In [1]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langgraph_supervisor import create_supervisor
import langchain
from tools.pubmed_search import pubmed


model = ChatOllama(
    model="qwen2.5:7b",           
    temperature=0,            
)

# Create specialized agents


pubmed_agent = create_agent(
    model=model,
    tools=[pubmed],
    name="pubmed_expert",
    system_prompt="You are a world class researcher with access to pubmed search. Find usefull sources for the patient case"
)

# screenfail_agent = create_agent(
#     model=model,
#     tools=[screenfail],
#     name="screenfail_expert",
#     system_prompt="You are a screenfail expert. Do not do any cgi clinics."
# )

# biomolecular_agent = create_agent(
#     model=model,
#     tools=[cgi_clinics, pubmed],
#     name="biomolecular_expert",
#     system_prompt="You are a world class biomolecular expert with access to CGI-Clinics. Always use one tool at a time. Find some clinical trials targeted the oncogenic mutation you find for the patient case"
# )

# Create supervisor workflow
workflow = create_supervisor(
    [pubmed_agent],
    model=model,
    system_prompt=(
        "You are a supervisor doctor managing a pubmed researcher in a moleculare tumor board in oncology."
        "For every question of the user, use pubmed_agent."
    )
)

# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": """I have this patient in MTB :  "Mrs. Brown, identified in our system as X11-01A, is a 49-year-old female patient who has been navigating her journey with metastatic colorectal cancer since 2019. 
            Her treatment history includes chemotherapy followed by surgical interventions, specifically a total mesorectal excision and a right-sided hemi-hepatectomy. In the previous year, a follow-up 
            CT scan revealed the presence of a single metastasis in the liver, prompting the initiation of a chemotherapy regimen that combined FOLFOX with Panitumumab. Currently, Mrs.
            Brown returns with a new CT scan dated February 10, 2024, ordered by her oncologist."
            I am asking you what is the best targeted therapeutical recommandation for this patient ? """
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

I have this patient in MTB :  "Mrs. Brown, identified in our system as X11-01A, is a 49-year-old female patient who has been navigating her journey with metastatic colorectal cancer since 2019. 
            Her treatment history includes chemotherapy followed by surgical interventions, specifically a total mesorectal excision and a right-sided hemi-hepatectomy. In the previous year, a follow-up 
            CT scan revealed the presence of a single metastasis in the liver, prompting the initiation of a chemotherapy regimen that combined FOLFOX with Panitumumab. Currently, Mrs.
            Brown returns with a new CT scan dated February 10, 2024, ordered by her oncologist."
            I am asking you what is the best targeted therapeutical recommandation for this patient ? 
================================== Ai Message ==================================
Name: supervisor

Given the information provided abo